In [14]:
from mlxtend.frequent_patterns import apriori,association_rules
import pandas as pd

In [55]:
warning_list = [['逾期','小贷过多','入不敷出'],
	        ['入不敷出','小贷过多','房租','低学历'],
	        ['房租','逾期','近期到期笔数过多','小贷过多'],
	        ['近期到期笔数过多','入不敷出','逾期','存在司法记录'],
	        ['房租','入不敷出','地址变动频繁','小贷过多']]
warning_df = pd.DataFrame(warning_list)

In [56]:
warning_df

,0,1,2,3
0,逾期,小贷过多,入不敷出,None
1,入不敷出,小贷过多,房租,低学历
2,房租,逾期,近期到期笔数过多,小贷过多
3,近期到期笔数过多,入不敷出,逾期,存在司法记录
4,房租,入不敷出,地址变动频繁,小贷过多


In [57]:
def deal(data):
	return data.dropna().tolist()
df_arr = warning_df.apply(deal,axis=1).tolist()	

In [58]:
df_arr

[['逾期', '小贷过多', '入不敷出'],
 ['入不敷出', '小贷过多', '房租', '低学历'],
 ['房租', '逾期', '近期到期笔数过多', '小贷过多'],
 ['近期到期笔数过多', '入不敷出', '逾期', '存在司法记录'],
 ['房租', '入不敷出', '地址变动频繁', '小贷过多']]

In [59]:
from mlxtend.preprocessing import TransactionEncoder	# 传入模型的数据需要满足特定的格式，可以用这种方法来转换为bool值，也可以用函数转换为0、1
 
te = TransactionEncoder()	# 定义模型
df_tf = te.fit_transform(df_arr)
# df_01 = df_tf.astype('int')			# 将 True、False 转换为 0、1 # 官方给的其它方法
# df_name = te.inverse_transform(df_tf)		# 将编码值再次转化为原来的商品名
df = pd.DataFrame(df_tf,columns=te.columns_)

In [60]:
df

,低学历,入不敷出,地址变动频繁,存在司法记录,小贷过多,房租,近期到期笔数过多,逾期
0,False,True,False,False,True,False,False,True
1,True,True,False,False,True,True,False,False
2,False,False,False,False,True,True,True,True
3,False,True,False,True,False,False,True,True
4,False,True,True,False,True,True,False,False


In [61]:
#按照0.5的支持度求频繁项集
frequent_itemsets = apriori(df,min_support=0.5,use_colnames=True)
frequent_itemsets.sort_values(by='support',ascending=False,inplace=True)

In [62]:
frequent_itemsets

,support,itemsets
0,0.8,(入不敷出)
1,0.8,(小贷过多)
2,0.6,(房租)
3,0.6,(逾期)
4,0.6,"(小贷过多, 入不敷出)"
5,0.6,"(小贷过多, 房租)"


In [63]:
association_rule = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.6)	# metric可以有很多的度量选项，返回的表列名都可以作为参数
association_rule.sort_values(by='leverage',ascending=False,inplace=True)    #关联规则可以按leverage排序
# print(association_rule)

In [64]:
association_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(小贷过多),(房租),0.8,0.6,0.6,0.75,1.2500,0.12,1.6
3,(房租),(小贷过多),0.6,0.8,0.6,1.00,1.2500,0.12,inf
0,(小贷过多),(入不敷出),0.8,0.8,0.6,0.75,0.9375,-0.04,0.8
1,(入不敷出),(小贷过多),0.8,0.8,0.6,0.75,0.9375,-0.04,0.8
